In [1]:
import math
import numpy as np

# T_k^0, R_k(last_t), A_k, r_k(last_t)
clients = []
ready_clients = []
T = 0.
A =  (500 * 5.5 / 4.5) + math.sqrt((5.5 / 3.5) * (500 / 4.5) ** 2)
l = 1 / (100 - math.sqrt(100))
r = 1.3 * 2 * (l * A)

In [2]:
import threading
import time
from IPython.display import clear_output
from threading import Thread

for i in range(100):
    T += 10
    A = (np.random.pareto(5.5) + 1) * 500
    rk = np.random.uniform(r * 0.7, r * 1.3)
    client = [T, np.random.uniform(1, A), A, rk]
    clients.append(client)

In [ ]:
e = threading.Event()
e1 = threading.Event()
e2 = threading.Event()

class TimeThread(Thread):
    def __init__(self):
        self.stopped = False
        Thread.__init__(self)
        
    def stop(self):
        self.stopped = True
    
    def run(self):
        global T
        while not self.stopped:
            T += 1
            e.wait(timeout=0.001)

class GenerationThread(Thread):
    def __init__(self, exp_param, power_param, val_power):
        self.exp_param = exp_param
        self.power_param = power_param
        self.val_power = val_power
        Thread.__init__(self)
    
    def run(self):
        global clients, e, T
        for i in range(150):
            pause = np.random.exponential(self.exp_param)
            e1.wait(timeout=pause / 1000.)
            A = (np.random.pareto(self.power_param) + 1) * self.val_power
            rk = np.random.uniform(r * 0.7, r * 1.3)
            client = [T, 0., A, rk]
            clients.append(client)
            

class ListeningThread(Thread):
    def __init__(self, timeout):
        self.timeout = timeout
        self.stopped = False
        Thread.__init__(self)
        
    def stop(self):
        self.stopped = True
     
    def run(self):
        global clients, T
        while not self.stopped:
            e2.wait(timeout=self.timeout)
            try:
                k = []
                for i in range(len(clients)):
                    Tk0 = clients[i][0]
                    Rk = clients[i][1] + 1e-20
                    clients[i][3] = np.random.uniform(r * 0.7, r*1.3)
                    rk = clients[i][3]
                    k.append(rk / (T-Tk0))
                # i = np.random.choice(np.arange(0, len(clients)), p=[ki / sum(k) for ki in k])
                i = k.index(max(k))
                clients[i][1] += min(clients[i][3], clients[i][2] - clients[i][1])
                clear_output(wait=True)
                print(len(clients))
                if clients[i][1] == clients[i][2]:
                    ready_clients.append(clients[i] + [T])
                    del clients[i]
            except:
                pass
        
time = TimeThread()
time.start()

listening = ListeningThread(0.00067)
listening.start()
        
generation = GenerationThread(100, 5.5, 500)
generation.start()

generation.join()
listening.stop()
time.stop()
clear_output(wait=True)
print("Ready!")
len(ready_clients)

78


In [4]:
HuaweiQ = 0.
for k in ready_clients:
    HuaweiQ += k[1] / (k[-1] - k[0])
HuaweiQ /= len(ready_clients)
HuaweiQ

4.544433350140282

$$Q_{Huawei}(Index_{Old}) = 0.69 $$

$$Q_{Huawei}(Index_{Intel, IBM}) = 2.84 $$

$$Q_{Huawei}(Index_{Huawei}) = 2.88 $$

$$Q_{Huawei}(Index_{Google}) = 3.08 $$

$$Q_{Huawei}(Index_{Timofey}) = 3.92 $$